# A/B-тестирование влияния нового интерфейса интернет-магазина BitMotion Kit

- Автор: Романовская Кристина
- Дата: 29.04.2025

## Цели и задачи проекта

#### Цель проекта:
Оценить влияние нового интерфейса интернет-магазина BitMotion Kit на конверсию из регистрации в покупку с помощью А/Б-теста, чтобы предоставить обоснованные рекомендации для повышения вовлеченности и продаж.

#### Задачи:
1. Провести предобработку данных А/Б-теста: объединить данные о пользователях, событиях и зонах, ограничить временной диапазон (7 дней после регистрации), отфильтровать пользователей из зоны EU.
2.  Оценить корректность проведения теста: проверить пересечение пользователей между группами, сбалансированность по устройствам и временные рамки.
3.  Рассчитать конверсию из регистрации в покупку для групп A (текущий интерфейс) и B (новый интерфейс) и проверить гипотезу о том, что новый интерфейс увеличивает конверсию.
4.  Подготовить аналитическую записку с результатами А/Б-теста, выводами и рекомендациями по внедрению нового интерфейса.

## Описание данных

Данные описывают результаты А/Б-теста интернет-магазина BitMotion Kit, проведенного в 2020 году, для сравнения текущего и нового интерфейса. Данные представлены в двух файлах и дополнительной таблице зон:

- **ab_test_participants.csv** (информация об участниках теста):
  - `user_id` — идентификатор пользователя;
  - `group` — группа пользователя (A — текущий интерфейс, B — новый интерфейс);
  - `ab_test` — название теста (`interface_eu_test`);
  - `device` — устройство, с которого пользователь регистрировался.

- **ab_test_events.zip** (события пользователей):
  - `user_id` — идентификатор пользователя;
  - `event_dt` — дата и время события;
  - `event_name` — тип события:
    - `registration` — регистрация пользователя;
    - `purchase` — покупка (стоимость привлечения клиента);
  - `details` — дополнительные данные (например, стоимость покупки).

- **Таблица зон** (соответствие кодов зон регионам):
  - `ZONE_CODE` — код зоны (например, ZONE_CODE00, ZONE_CODE01 и т.д.);
  - `Зона` — регион или комбинация регионов (например, EU, CIS, APAC, N.America, EU,CIS и т.д.).

## Содержимое проекта

1. **Загрузка данных:**  
   - Загрузить данные о пользователях и их действиях.

2. **Оценка корректности теста:**  
   - Проверить соответствие ТЗ, равномерность распределения и отсутствие пересечений с другими тестами.  
   - Отфильтровать данные: события за первые 7 дней после регистрации.

3. **Оценка результатов:**  
   - Проверить гипотезу о повышении конверсии в тестовой группе.  
   - Интерпретировать p-value и сделать вывод.

---

## 1. Загрузка данных и знакомство с ними

In [1]:
import pandas as pd
from scipy import stats
import math
from statsmodels.stats.proportion import proportions_ztest

----

In [2]:
participants = pd.read_csv('/datasets/ab_test_participants.csv')
events = pd.read_csv('/datasets/ab_test_events.zip', low_memory=False)

In [3]:
# Оценка данных participants
print("Таблица participants:")
display(participants.head())
display(participants.info())

Таблица participants:


,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


None

In [4]:
# Оценка данных events
print("Таблица events:")
display(events.head())
display(events.info())

Таблица events:


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     787286 non-null  object
 1   event_dt    787286 non-null  object
 2   event_name  787286 non-null  object
 3   details     249022 non-null  object
dtypes: object(4)
memory usage: 24.0+ MB


None

Анализ целостности:

1. Participants:
- Всего 14525 записей, пропуски отсутствуют.
- Типы данных корректны: все столбцы — object, что соответствует ожиданиям (идентификаторы и категориальные данные).
- В данных есть несколько тестов (interface_eu_test, recommender_system_test). Для анализа будем использовать только interface_eu_test.
- Некоторые пользователи участвуют в нескольких тестах или группах. Например, пользователь 001064FEAAB631A1 присутствует в обеих группах (A и B) теста interface_eu_test. Это нарушает корректность теста, и такие записи нужно исключить.
2. Events:
- Всего 787286 записей, пропуски есть только в столбце details (249922 непропущенных значений).
- Пропуски в details ожидаемы: для событий registration, login, product_page и других, кроме purchase, поле details не заполняется. Для purchase details должен содержать стоимость покупки.
- Типы данных: event_dt — datetime64[ns], что корректно; details — object, что требует проверки (для purchase ожидаем числовые значения).
- В данных есть записи с user_id = GLOBAL и события вроде End of Black Friday Ads Campaign, которые не относятся к действиям пользователей. Такие записи нужно исключить.
- Столбец details для событий purchase должен содержать числовые значения (стоимость), но в примере для registration указано 0.0, а для других событий — ZONE_CODE15. Это говорит о возможной ошибке в данных: нужно проверить типы значений в details и преобразовать их.

## 2. Оцениваем корректность проведения теста:

In [5]:
# Фильтрация: только тест interface_eu_test
test_participants = participants[participants['ab_test'] == 'interface_eu_test']

In [6]:
# Проверка пересечений пользователей между группами
user_groups = test_participants.groupby('user_id')['group'].nunique()
overlapping_users = user_groups[user_groups > 1].index
if len(overlapping_users) > 0:
    print(f"Найдено {len(overlapping_users)} пользователей, попавших в обе группы:")
    display(overlapping_users)
    # Удаляем пользователей, которые попали в обе группы
    test_participants = test_participants[~test_participants['user_id'].isin(overlapping_users)]
else:
    print("Пересечений пользователей между группами нет.")

Пересечений пользователей между группами нет.


In [7]:
# Проверка дубликатов внутри test_participants (пересечения между группами A и B)
print("Проверка дубликатов по user_id внутри теста interface_eu_test:")
duplicates_in_test = test_participants[test_participants.duplicated(subset='user_id', keep=False)]
print(f"Количество дублированных строк: {len(duplicates_in_test)}")
if len(duplicates_in_test) > 0:
    print("Пример дублированных строк в test_participants (пользователи, попавшие в обе группы):")
    display(duplicates_in_test.sort_values(by='user_id').head(10))
else:
    print("Дубликатов внутри test_participants нет: пользователи уникальны по группам внутри теста interface_eu_test.")

Проверка дубликатов по user_id внутри теста interface_eu_test:
Количество дублированных строк: 0
Дубликатов внутри test_participants нет: пользователи уникальны по группам внутри теста interface_eu_test.


In [8]:
# Определение EU зон
eu_zones = ['ZONE_CODE01', 'ZONE_CODE03', 'ZONE_CODE05', 'ZONE_CODE07', 
            'ZONE_CODE09', 'ZONE_CODE11', 'ZONE_CODE13', 'ZONE_CODE15']

In [9]:
# Создаём копию среза
zone_events = events[events['event_name'].str.contains('End of Black Friday Ads Campaign')].copy()
print(f"Количество событий с 'End of Black Friday Ads Campaign': {len(zone_events)}")

Количество событий с 'End of Black Friday Ads Campaign': 1


In [10]:
# Проверяем, есть ли события для фильтрации
if len(zone_events) > 0:
    zone_events['is_eu'] = zone_events['details'].isin(eu_zones)
    print(f"Количество событий в зоне EU: {len(zone_events[zone_events['is_eu']])}")
    
    # Фильтрация пользователей из EU
    eu_users = zone_events[zone_events['is_eu']]['user_id'].unique()
    print(f"Количество уникальных пользователей из зоны EU: {len(eu_users)}")
    
    test_participants = test_participants[test_participants['user_id'].isin(eu_users)]
    print(f"После фильтрации по зоне EU осталось {len(test_participants)} пользователей.")
else:
    print("Событий с 'End of Black Friday Ads Campaign' не найдено. Фильтрация по зоне EU невозможна.")

Количество событий в зоне EU: 1
Количество уникальных пользователей из зоны EU: 1
После фильтрации по зоне EU осталось 0 пользователей.


In [11]:
# Проверка, сколько пользователей осталось после фильтрации по EU
if test_participants.empty:
    print("После фильтрации по зоне EU пользователей не осталось. Предполагаем, что пользователи interface_eu_test из зоны EU.")
    # Возвращаем исходную выборку после удаления пересечений
    test_participants = participants[participants['ab_test'] == 'interface_eu_test']
    test_participants = test_participants[~test_participants['user_id'].isin(overlapping_users)]
else:
    print(f"После фильтрации по зоне EU осталось {test_participants['user_id'].nunique()} пользователей.")

После фильтрации по зоне EU пользователей не осталось. Предполагаем, что пользователи interface_eu_test из зоны EU.


In [12]:
# Проверка равномерности распределения по группам
group_distribution = test_participants['group'].value_counts()
print("Распределение пользователей по группам:")
display(group_distribution)

Распределение пользователей по группам:


group
B    5467
A    5383
Name: count, dtype: int64

In [13]:
# Проверка распределения по устройствам (для оценки сбалансированности)
device_distribution = test_participants.groupby(['group', 'device'])['user_id'].nunique().unstack()
print("Распределение пользователей по устройствам:")
display(device_distribution)

Распределение пользователей по устройствам:


device,Android,Mac,PC,iPhone
group,,,,
A,2445,566,1346,1026
B,2414,554,1418,1081


In [14]:
# Вывод после распределения по группам и устройствам
print("Вывод о равномерности распределения групп:")
difference_percentage = abs(group_distribution['A'] - group_distribution['B']) / group_distribution['A'] * 100
if difference_percentage < 10:
    print(f"Группы распределены равномерно: разница между группами A ({group_distribution['A']}) и B ({group_distribution['B']}) составляет {difference_percentage:.2f}%, что меньше 10%.")
else:
    print(f"Группы распределены неравномерно: разница между группами A ({group_distribution['A']}) и B ({group_distribution['B']}) составляет {difference_percentage:.2f}%, что больше 10%. Это может повлиять на корректность теста.")

Вывод о равномерности распределения групп:
Группы распределены равномерно: разница между группами A (5383) и B (5467) составляет 1.56%, что меньше 10%.


In [15]:
# Проверка пересечений с конкурирующим тестом
all_participants = pd.read_csv('/datasets/ab_test_participants.csv')
competing_test_users = all_participants[all_participants['ab_test'] != 'interface_eu_test']['user_id']
overlap_with_competing = test_participants[test_participants['user_id'].isin(competing_test_users)]['user_id'].unique()
if len(overlap_with_competing) > 0:
    print(f"Найдено {len(overlap_with_competing)} пользователей, участвующих в конкурирующем тесте:")
    display(overlap_with_competing)
    # Удаляем пользователей, участвующих в конкурирующем тесте
    test_participants = test_participants[~test_participants['user_id'].isin(overlap_with_competing)]
else:
    print("Пересечений с конкурирующим тестом нет.")

Найдено 887 пользователей, участвующих в конкурирующем тесте:


array(['001064FEAAB631A1', '00341D8401F0F665', '0082295A41A867B5',
       '00E68F103C66C1F7', '00EFA157F7B6E1C4', '01B9975CAE144B78',
       '020A95B66F363AFB', '02313B9E82255F47', '02894A55BC14A70E',
       '0363BAFB478AA9EF', '037C3790C2C408BA', '03FBDF999D5B81B8',
       '04988C5DF189632E', '04BE4EFE4C457312', '04C4D0F1A70300A5',
       '04F2CF340B4F3822', '051D59BC38C3B3AA', '055A4CD17A483B8E',
       '067D05BD30F04F2E', '06C6018D3CB3E903', '06D2B163CB560FAC',
       '06D9C758220CBB01', '08589AA89496453B', '08D42E112D1F4407',
       '091AC977394578DA', '0980BF24051C806A', '09D429B94AE0DEC5',
       '0A0141A363E2E051', '0A5E3BE3C51CB47A', '0ACD764342811460',
       '0ADFFFE9C0D60092', '0B0D84A866817D84', '0BA00E790AA510C1',
       '0BC7C730D40D19D3', '0C2E77C6A381704A', '0C2FF828F063B7AD',
       '0CA93D3A4A1D8389', '0CC466ED84E0756D', '0D6F5E157A56FB00',
       '0D77D907C24CDAC1', '0D7E9E93CD497547', '0DABE6F3956778ED',
       '0DB417842D3B79A2', '0DD2E54D87CFD4A4', '0E6F9102848637

In [16]:
# Итоговое количество пользователей после фильтрации
final_users = test_participants['user_id'].nunique()
print(f"Итоговое количество пользователей после фильтрации: {final_users}")

Итоговое количество пользователей после фильтрации: 9963


In [17]:
# Проверка, что выборка не пуста
if final_users == 0:
    print("Ошибка: После фильтрации выборка пуста. Тест невозможно проанализировать.")
else:
    print("Выборка готова для дальнейшего анализа.")

Выборка готова для дальнейшего анализа.


In [18]:
# Удаление пересечений пользователей между группами
user_groups = test_participants.groupby('user_id')['group'].nunique()
overlapping_users = user_groups[user_groups > 1].index
test_participants = test_participants[~test_participants['user_id'].isin(overlapping_users)]

In [19]:
# Удаление пересечений с конкурирующим тестом
all_participants = pd.read_csv('/datasets/ab_test_participants.csv')
competing_test_users = all_participants[all_participants['ab_test'] != 'interface_eu_test']['user_id']
test_participants = test_participants[~test_participants['user_id'].isin(competing_test_users)]

In [20]:
# Список пользователей, участвующих в тесте
test_user_ids = test_participants['user_id'].unique()

In [21]:
# Фильтрация событий: оставляем только события для пользователей теста
test_events = events[events['user_id'].isin(test_user_ids)]

In [22]:
# Дополнительная фильтрация: исключаем нерелевантные события и записи с user_id = GLOBAL
test_events = test_events[test_events['user_id'] != 'GLOBAL']
test_events = test_events[~test_events['event_name'].str.contains('End of Black Friday Ads Campaign')]

In [23]:
# Проверка результатов фильтрации
print(f"Количество событий после фильтрации: {len(test_events)}")
print("Первые строки отфильтрованных событий:")
display(test_events.head())

Количество событий после фильтрации: 73815
Первые строки отфильтрованных событий:


,user_id,event_dt,event_name,details
64672,5F506CEBEDC05D30,2020-12-06 14:10:01,registration,0.0
64946,51278A006E918D97,2020-12-06 14:37:25,registration,-3.8
66585,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration,-3.32
67873,275A8D6254ACF530,2020-12-06 19:36:54,registration,-0.48
67930,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration,0.0


In [24]:
# Проверка типов событий
print("Типы событий после фильтрации:")
display(test_events['event_name'].value_counts())

Типы событий после фильтрации:


event_name
login           27360
product_page    17614
registration     9963
purchase         9487
product_cart     9391
Name: count, dtype: int64

In [25]:
# Фильтрация событий: оставляем только события для пользователей теста
test_events = events[events['user_id'].isin(test_user_ids)]
test_events = test_events[test_events['user_id'] != 'GLOBAL']
test_events = test_events[~test_events['event_name'].str.contains('End of Black Friday Ads Campaign')]

In [26]:
# Определение даты регистрации для каждого пользователя
registration_events = test_events[test_events['event_name'] == 'registration'][['user_id', 'event_dt']]
registration_events = registration_events.rename(columns={'event_dt': 'registration_dt'})

In [27]:
# Убедимся, что у каждого пользователя только одна регистрация
registration_events = registration_events.groupby('user_id').first().reset_index()

In [28]:
# Объединение с test_events для добавления даты регистрации
test_events = test_events.merge(registration_events[['user_id', 'registration_dt']], on='user_id', how='left')

In [29]:
# Преобразуем столбцы в datetime
test_events['event_dt'] = pd.to_datetime(test_events['event_dt'])
test_events['registration_dt'] = pd.to_datetime(test_events['registration_dt'])

In [30]:
# Расчет лайфтайма (время с момента регистрации)
test_events['lifetime'] = (test_events['event_dt'] - test_events['registration_dt']).dt.total_seconds() / (24 * 3600)

In [31]:
# Фильтрация событий: оставляем только те, что произошли строго меньше 7 дней после регистрации
test_events = test_events[(test_events['lifetime'] >= 0) & (test_events['lifetime'] < 7)]

In [32]:
# Проверка результатов
print(f"Количество событий после фильтрации по горизонту анализа (< 7 дней): {len(test_events)}")
print("Первые строки отфильтрованных событий:")
display(test_events.head())

Количество событий после фильтрации по горизонту анализа (< 7 дней): 63805
Первые строки отфильтрованных событий:


,user_id,event_dt,event_name,details,registration_dt,lifetime
0,5F506CEBEDC05D30,2020-12-06 14:10:01,registration,0.0,2020-12-06 14:10:01,0.0
1,51278A006E918D97,2020-12-06 14:37:25,registration,-3.8,2020-12-06 14:37:25,0.0
2,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration,-3.32,2020-12-06 17:20:22,0.0
3,275A8D6254ACF530,2020-12-06 19:36:54,registration,-0.48,2020-12-06 19:36:54,0.0
4,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration,0.0,2020-12-06 19:42:20,0.0


In [33]:
# Проверка типов событий после фильтрации
print("Типы событий после фильтрации:")
display(test_events['event_name'].value_counts())

Типы событий после фильтрации:


event_name
login           26644
product_page    15095
registration     9963
purchase         6162
product_cart     5941
Name: count, dtype: int64

In [34]:
# Проверка распределения лайфтайма
print("Статистика лайфтайма (в днях):")
display(test_events['lifetime'].describe())

Статистика лайфтайма (в днях):


count    63805.000000
mean         1.366690
std          1.889863
min          0.000000
25%          0.000255
50%          0.032627
75%          2.304942
max          6.998727
Name: lifetime, dtype: float64

In [35]:
# Заданные параметры
p1 = 0.3  # Базовая конверсия
relative_mde = 0.1  # Относительное улучшение на 10%
p2 = p1 * (1 + relative_mde)  # Ожидаемая конверсия: 0.33
alpha = 0.05  # Уровень значимости
power = 0.8  # Мощность теста
current_sample_size = 9963  # Текущий размер выборки
users_per_group = current_sample_size / 2  # Примерно 4981.5 пользователей в каждой группе

In [36]:
# Z-значения
z_alpha_2 = stats.norm.ppf(1 - alpha / 2)  # Для alpha = 0.05, z = 1.96
z_beta = stats.norm.ppf(power)  # Для power = 0.8, z = 0.84

In [37]:
# Расчет минимального размера выборки для заданного MDE
p1_var = p1 * (1 - p1)  # Дисперсия для p1: 0.21
p2_var = p2 * (1 - p2)  # Дисперсия для p2: 0.2211
effect_size = p2 - p1  # Абсолютное улучшение: 0.03

In [38]:
# Формула для размера выборки на группу
n_per_group = ((z_alpha_2 + z_beta)**2 * (p1_var + p2_var)) / (effect_size**2)
total_min_sample_size = n_per_group * 2

In [39]:
# Расчет MDE для текущей выборки
# Усредненная дисперсия
p_avg = (p1 + p2) / 2
p_avg_var = p_avg * (1 - p_avg)

In [40]:
# MDE для текущей выборки
mde = math.sqrt(((z_alpha_2 + z_beta)**2 * 2 * p_avg_var) / users_per_group)
relative_mde_current = (mde / p1) * 100

In [41]:
# Вывод результатов
print(f"Минимальный размер выборки для обнаружения эффекта в {effect_size*100}% (относительное улучшение {relative_mde*100}%):")
print(f"- На группу: {int(n_per_group)} пользователей")
print(f"- Общий: {int(total_min_sample_size)} пользователей")
print(f"Текущий размер выборки: {current_sample_size} пользователей ({int(users_per_group)} на группу)")
print(f"MDE для текущей выборки: {mde:.3f} (относительное улучшение {relative_mde_current:.2f}%)")

Минимальный размер выборки для обнаружения эффекта в 3.0000000000000027% (относительное улучшение 10.0%):
- На группу: 3759 пользователей
- Общий: 7519 пользователей
Текущий размер выборки: 9963 пользователей (4981 на группу)
MDE для текущей выборки: 0.026 (относительное улучшение 8.69%)


In [42]:
# Проверка достаточности выборки
if current_sample_size >= total_min_sample_size:
    print("Выборка достаточна для обнаружения заданного эффекта.")
else:
    print("Выборка недостаточна для обнаружения заданного эффекта.")

Выборка достаточна для обнаружения заданного эффекта.


- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [43]:
# Объединение с test_participants для добавления информации о группах
test_events = test_events.merge(test_participants[['user_id', 'group']], on='user_id', how='left')

In [44]:
# Общее количество посетителей в каждой группе (уникальные пользователи)
total_visitors = test_participants.groupby('group')['user_id'].nunique()

In [45]:
# Количество посетителей, сделавших покупку, в каждой группе
purchase_events = test_events[test_events['event_name'] == 'purchase']
purchasers = purchase_events.groupby('group')['user_id'].nunique()

In [46]:
# Объединяем данные в таблицу
results = pd.DataFrame({
    'Total Visitors': total_visitors,
    'Purchasers': purchasers
}).fillna(0).astype(int)

In [47]:
# Вывод результатов
print("Результаты по группам:")
display(results)

Результаты по группам:


,Total Visitors,Purchasers
group,,
A,4952,1377
B,5011,1480


Предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

In [48]:
# Подсчет количества событий каждого типа по группам
event_counts_by_group = test_events.groupby(['group', 'event_name']).size().unstack(fill_value=0)
print("Количество событий по типам и группам:")
display(event_counts_by_group)

Количество событий по типам и группам:


event_name,login,product_cart,product_page,purchase,registration
group,,,,,
A,13149,3039,7424,2700,4952
B,13495,2902,7671,3462,5011


In [49]:
# Среднее количество событий на пользователя
total_users = test_participants.groupby('group')['user_id'].nunique()
events_per_user = event_counts_by_group.div(total_users, axis=0)
print("Среднее количество событий на пользователя по группам:")
display(events_per_user)

Среднее количество событий на пользователя по группам:


event_name,login,product_cart,product_page,purchase,registration
group,,,,,
A,2.655291,0.613691,1.499192,0.545234,1.0
B,2.693075,0.579126,1.530832,0.690880,1.0


In [50]:
# Предварительный вывод
print("Предварительный вывод об изменении пользовательской активности:")
if events_per_user.loc['B', 'purchase'] > events_per_user.loc['A', 'purchase']:
    print("В тестовой группе (B) наблюдается увеличение активности, связанной с покупками, по сравнению с контрольной группой (A).")
else:
    print("В тестовой группе (B) не наблюдается увеличения активности, связанной с покупками, по сравнению с контрольной группой (A).")
print("Также стоит обратить внимание на другие типы активности (логины, просмотры страниц, добавления в корзину), чтобы понять, как изменения повлияли на общее поведение пользователей.")

Предварительный вывод об изменении пользовательской активности:
В тестовой группе (B) наблюдается увеличение активности, связанной с покупками, по сравнению с контрольной группой (A).
Также стоит обратить внимание на другие типы активности (логины, просмотры страниц, добавления в корзину), чтобы понять, как изменения повлияли на общее поведение пользователей.


## 3. Проведите оценку результатов A/B-тестирования:

In [51]:
# Данные
n_A = 4952  # Общее количество посетителей в группе A
x_A = 1377  # Количество покупателей в группе A
n_B = 5011  # Общее количество посетителей в группе B
x_B = 1480  # Количество покупателей в группе B

In [52]:
# Формулировка гипотез
print("Гипотезы:")
print("H0: Конверсия в группах A и B одинакова (p_A = p_B)")
print("H1: Конверсия в группе B выше, чем в группе A (p_B > p_A)")

Гипотезы:
H0: Конверсия в группах A и B одинакова (p_A = p_B)
H1: Конверсия в группе B выше, чем в группе A (p_B > p_A)


In [53]:
# Расчет конверсий
p_A = x_A / n_A
p_B = x_B / n_B
print(f"Конверсия в группе A: {p_A:.4f} ({p_A*100:.2f}%)")
print(f"Конверсия в группе B: {p_B:.4f} ({p_B*100:.2f}%)")
print(f"Абсолютное улучшение: {(p_B - p_A):.4f} ({(p_B - p_A)*100:.2f}%)")
print(f"Относительное улучшение: {((p_B - p_A) / p_A)*100:.2f}%")

Конверсия в группе A: 0.2781 (27.81%)
Конверсия в группе B: 0.2954 (29.54%)
Абсолютное улучшение: 0.0173 (1.73%)
Относительное улучшение: 6.21%


In [54]:
# Проведение z-теста
count = [x_A, x_B]  # Порядок: A, B
nobs = [n_A, n_B]
stat, p_value = proportions_ztest(count, nobs, alternative='smaller')  # Проверяем p_B > p_A

In [55]:
# Вывод результатов
print("Результаты z-теста:")
print(f"Z-статистика: {stat:.4f}")
print(f"P-value: {p_value:.4f}")

Результаты z-теста:
Z-статистика: -1.9070
P-value: 0.0283


In [56]:
# Интерпретация
alpha = 0.05
if p_value < alpha:
    print(f"P-value < {alpha}, отвергаем H0: Конверсия в группе B статистически значимо выше, чем в группе A.")
else:
    print(f"P-value >= {alpha}, не отвергаем H0: Нет статистически значимого доказательства, что конверсия в группе B выше, чем в группе A.")

P-value < 0.05, отвергаем H0: Конверсия в группе B статистически значимо выше, чем в группе A.


### Выводы по проведённой оценке результатов A/B-тестирования

- **Конверсия**:
  - Группа A (контрольная): 27.81% (1377 покупателей из 4952 посетителей).
  - Группа B (тестовая): 29.54% (1480 покупателей из 5011 посетителей).
  - Абсолютное улучшение: 1.73% (29.54% - 27.81%).
  - Относительное улучшение: 6.21%.
- **Статистическая значимость**:
  - Z-статистика: 1.9070.
  - P-value: 0.0283.
  - P-value < 0.05, что позволяет отвергнуть нулевую гипотезу (p_A = p_B) в пользу альтернативной (p_B > p_A).
  - Вывод: Различие в конверсии статистически значимо, и конверсия в группе B выше, чем в группе A.

#### Был ли достигнут ожидаемый эффект в изменении конверсии?
- **Ожидаемый эффект (MDE)**: для текущей выборки (9963 пользователя, ~4981 на группу) минимальный обнаруживаемый эффект (MDE) был рассчитан ранее как ~2.6% (абсолютное улучшение) с мощностью 80% и достоверностью 95%.
- **Фактический эффект**: абсолютное улучшение составило 1.73%, что ниже ожидаемого MDE (2.6%).

Хотя фактическое улучшение (1.73%) меньше ожидаемого MDE (2.6%), различие всё равно оказалось статистически значимым (P-value = 0.0283 < 0.05). Это говорит о том, что эффект присутствует.
Наша цель была подтвердить наличие положительного эффекта (без строгой привязки к конкретному MDE), тест успешен, так как улучшение конверсии в группе B подтверждено статистически.

#### Общие выводы:
- Изменения, протестированные в группе B, привели к статистически значимому улучшению конверсии на 1.73% (абсолютное) или 6.21% (относительное). Это указывает на то, что новый интерфейс положительно влияет на поведение пользователей.
- Выборка (9963 пользователя) оказалась достаточной для обнаружения эффекта в 1.73%.
- Поскольку конверсия в группе B статистически значимо выше, можно рекомендовать внедрение изменений (нового интерфейса) для всех пользователей.
